### launching of SPs



In [1]:
from datetime import timedelta
from decimal import Decimal
from tqdm import tqdm
import pandas as pd

import pyodbc
from sql.config import SQL_SERVER, DB_NAME
import sql.naming_convention as nc
from sql.sql_requests import SQL_Communicator

In [2]:
def format_time_delta(duration: timedelta):
    # Get the total seconds
    total_seconds = duration.total_seconds()
    # days = duration.days
    hours = duration.seconds // 3600  # Convert seconds to hours
    minutes = (duration.seconds // 60) % 60  # Convert remaining seconds to minutes
    seconds = duration.seconds % 60
    if hours > 0:
        return f'{hours:02d}:{minutes:02d}:{int(seconds):02d}'
    if minutes > 0:
        return f'{minutes:02d}:{int(seconds):02d}'
    return f'{total_seconds:4.2f}'

In [3]:
entity_names =[
'CompanyRel_CompanyRel',
'Competitors_Competitors',
'Competitors_CompetitorToSource',
'PeopleBasic_Person',
'PeopleBasic_ProToProFunction',
'CompanyNames_AlternateCompanyName',
'CompanyNames_SearchCompanyName',
'KeyDocument_KeyDocumentToSecurity',
'FinAuditor_FinancialToAdvisor',
'BusinessRel_BusinessRel',
'Product_Product',
'KeyDocument_KeyDocument',
'PeopleBasic_Professional',
'CompanyNames_NativeCompanyName'
]

durations, cnt = [], 0
with SQL_Communicator() as sc:
    for en in tqdm(entity_names):
        cnt += 1
        print(f'flow for target entity {en}..')
        en2 = nc.default_rename(en)
        sp_names = [nc.pull_sp_name(en2), nc.merge_sp_name(en2)]
        stage_tbl = nc.stg_table_name(en)
        trg_tbl = nc.table_name(nc.default_rename(en))
        ent_durs, flow_td = [en2], timedelta()  
        for i, sp_name in enumerate(sp_names):
            exec_stm = f"EXEC {sp_name}"
            print(f'- {exec_stm}...')
            try:
                td = sc.get_execution_metrics(exec_stm)
                ent_durs.append(format_time_delta(td))
                if i < 2:
                    flow_td +=td
                if  i == 1:
                    ent_durs.append(format_time_delta(flow_td))
                print(f'  - duration {format_time_delta(td)}')
                if i == 0:  ## pull sp
                    rows_count, table_size = sc.get_table_size(stage_tbl)
                    print(f'rows {rows_count}, stage table size: {table_size}')
                # sc.get_execution_metrics(f'TRUNCATE table {trg_tbl}')
            except Exception as ex:
                print(f'shit happened {ex}')
        rows_count = rows_count /1000
        velocity = table_size / Decimal(flow_td.total_seconds()) if flow_td.total_seconds() else 0 
        ent_durs.extend([f'{rows_count:7.2f}' , f'{table_size:7.2f}', f'{velocity:7.2f}'])
        durations.append(ent_durs)
        if cnt == len(entity_names):
            print('done')
sc = None
        # ent_durs.append(sum(ent_durs))


connecting...
Connection successful!


  0%|          | 0/14 [00:00<?, ?it/s]

flow for target entity CompanyRel_CompanyRel..
- EXEC dbo.PullData_CompanyRel_CompanyRel2_prc...
  - duration 01:30
rows 13406763, stage table size: 653.203125
- EXEC dbo.MergeData_CompanyRel_CompanyRel2_prc...


  7%|▋         | 1/14 [02:30<32:42, 150.95s/it]

  - duration 01:00
flow for target entity Competitors_Competitors..
- EXEC dbo.PullData_Competitors_Competitors2_prc...
  - duration 2.58
rows 920338, stage table size: 31.453125
- EXEC dbo.MergeData_Competitors_Competitors2_prc...


...

  - duration 36.53
flow for target entity PeopleBasic_Professional..
- EXEC dbo.PullData_PeopleBasic_Professional2_prc...
  - duration 01:17
rows 10688109, stage table size: 829.335937
- EXEC dbo.MergeData_PeopleBasic_Professional2_prc...


 93%|█████████▎| 13/14 [39:28<02:21, 141.83s/it]

  - duration 01:30
flow for target entity CompanyNames_NativeCompanyName..
- EXEC dbo.PullData_CompanyNames_NativeCompanyName2_prc...
  - duration 2.51
rows 285343, stage table size: 36.453125
- EXEC dbo.MergeData_CompanyNames_NativeCompanyName2_prc...


100%|██████████| 14/14 [39:32<00:00, 169.43s/it]

  - duration 1.39
done


In [4]:
columns = ['Entity name', 'Pull SP', 'Merge SP', 'sum',  'rows_count, K', 'table_size, MB', 'velocity, MB/sec']
df = pd.DataFrame(durations, columns = columns)
# df['table_size, MB'] = df ['table_size, MB'].astype('int')
# df['rows_count, K'] = df['rows_count, K']/1000
df

,Entity name,Pull SP,Merge SP,sum,"rows_count, K","table_size, MB","velocity, MB/sec"
0,CompanyRel_CompanyRel2,01:30,01:00,02:30,13406.76,653.20,4.33
1,Competitors_Competitors2,2.58,4.28,6.85,920.34,31.45,4.59
2,Competitors_CompetitorToSource2,3.00,4.35,7.34,920.39,31.45,4.28
3,PeopleBasic_Person2,27.25,41.13,01:08,5959.98,321.14,4.70
4,PeopleBasic_ProToProFunction2,01:36,02:37,04:14,15095.86,939.70,3.69
5,CompanyNames_AlternateCompanyName2,15.14,13.69,28.83,2676.60,202.70,7.03
6,CompanyNames_SearchCompanyName2,11:27,14:31,25:59,161240.44,10684.40,6.85
7,KeyDocument_KeyDocumentToSecurity2,13.12,1.39,14.51,886.61,79.14,5.45
8,FinAuditor_FinancialToAdvisor2,3.20,6.67,9.87,1229.24,47.95,4.86
9,BusinessRel_BusinessRel2,1.23,47.95,0.00,None,None,None
